<a href="https://colab.research.google.com/github/VicDc/VIC_/blob/OPIT/8003_Smart_Home_AI_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Setup & Config (!)**

In [4]:
!pip install transformers torch -q
print("Dependency check completed/skipped.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.9 MB/s eta 0:00:00
Dependency check completed/skipped.


# 1 -Device Definitions

In [6]:
import re  # For regular expressions in the parser
import sys # For system interaction (though not heavily used here)

print("Standard imports completed.")

# --- Light Class Definition ---
class Light:
    """
    Simulates a smart light that can be turned on or off.
    """
    def __init__(self, name="Living Room Light"):
        """
        Initializes the light.
        Args:
            name (str): The identifying name of the light.
        """
        self.name = name
        self.is_on = False  # Initial state: off

    def turn_on(self):
        """Turns the light on if it's off."""
        if not self.is_on:
            self.is_on = True
            return f"{self.name} is now ON."
        return f"{self.name} is already ON."

    def turn_off(self):
        """Turns the light off if it's on."""
        if self.is_on:
            self.is_on = False
            return f"{self.name} is now OFF."
        return f"{self.name} is already OFF."

    def get_status(self):
        """Returns the current status of the light (ON/OFF)."""
        state = "ON" if self.is_on else "OFF"
        return f"{self.name}: {state}"

# --- Fan Class Definition ---
class Fan:
    """
    Simulates a smart fan with multiple speeds (low, medium, high)
    and an on/off state.
    """
    # Valid speeds for the fan
    VALID_SPEEDS = ["low", "medium", "high"]

    def __init__(self, name="Ceiling Fan"):
        """
        Initializes the fan.
        Args:
            name (str): The identifying name of the fan.
        """
        self.name = name
        self.is_on = False  # Initial state: off
        self.speed = "low" # Default speed when turned on

    def turn_on(self):
        """Turns the fan on (to its last set speed or default 'low')."""
        if not self.is_on:
            self.is_on = True
            return f"{self.name} is now ON (Speed: {self.speed})."
        return f"{self.name} is already ON."

    def turn_off(self):
        """Turns the fan off."""
        if self.is_on:
            self.is_on = False
            return f"{self.name} is now OFF."
        return f"{self.name} is already OFF."

    def set_speed(self, speed):
        """
        Sets the fan speed.
        If the fan is off, it turns it on when setting the speed.
        Args:
            speed (str): The desired speed ("low", "medium", "high").
        Returns:
            str: Feedback message about the operation.
        """
        speed_lower = speed.lower() # Convert to lowercase for comparison
        if speed_lower in self.VALID_SPEEDS:
            self.speed = speed_lower
            feedback = f"{self.name} speed set to {self.speed}."
            # If setting speed while fan is off, turn it on
            if not self.is_on:
                self.is_on = True
                feedback = f"{self.name} turned ON and speed set to {self.speed}."
            return feedback
        else:
            # Error message if the speed is invalid
            valid_options = ', '.join(self.VALID_SPEEDS)
            return f"Invalid speed '{speed}'. Valid speeds are: {valid_options}."

    def get_status(self):
        """Returns the current status of the fan (ON/OFF and speed)."""
        state = "ON" if self.is_on else "OFF"
        status = f"{self.name}: {state}"
        if self.is_on:
            status += f" (Speed: {self.speed})"
        return status

# --- Thermostat Class Definition ---
class Thermostat:
    """
    Simulates a smart thermostat that allows setting the temperature
    within a defined range.
    """
    MIN_TEMP = 18  # Minimum settable temperature
    MAX_TEMP = 30  # Maximum settable temperature

    def __init__(self, name="Main Thermostat", current_temp=22):
        """
        Initializes the thermostat.
        Args:
            name (str): The identifying name of the thermostat.
            current_temp (int): The desired initial temperature.
        """
        self.name = name
        # Ensure the initial temperature is within the valid bounds
        self._temperature = max(self.MIN_TEMP, min(self.MAX_TEMP, current_temp))

    def set_temperature(self, temp):
        """
        Sets the thermostat temperature.
        Args:
            temp (int or str): The desired temperature (will be converted to int).
        Returns:
            str: Feedback message about the operation.
        """
        try:
            temp_int = int(temp) # Convert input to integer
            # Check if the temperature is within the valid range
            if self.MIN_TEMP <= temp_int <= self.MAX_TEMP:
                self._temperature = temp_int
                return f"{self.name} temperature set to {self._temperature}°C."
            else:
                # Message if the temperature is out of range
                return (f"Invalid temperature '{temp}'. "
                        f"Must be between {self.MIN_TEMP}°C and {self.MAX_TEMP}°C.")
        except ValueError:
            # Message if the input is not a valid number
            return f"Invalid temperature format '{temp}'. Please provide a number."

    def get_status(self):
        """Returns the current temperature setting."""
        return f"{self.name}: Set to {self._temperature}°C"

print("Device class definitions completed.")


Standard imports completed.
Device class definitions completed.


In [7]:
# -----------------------------------------------------------------------------
# Block 2: Parsing Logic and AI Loading (from parser.py)
# -----------------------------------------------------------------------------

# Attempt to import the Hugging Face Transformers library
try:
    from transformers import AutoTokenizer, AutoModel
    transformers_available = True # Flag to indicate if the library is available
except ImportError:
    # Warning if the library is not installed
    print("\nWarning: 'transformers' library not found.")
    transformers_available = False

# --- AI Model Loading (as requested) ---

model_name = "Musixmatch/umberto-wikipedia-uncased-v1" # Specified model name
tokenizer = None # Variable for the tokenizer
model = None     # Variable for the model

if transformers_available:
    try:
        print(f"\nLoading Hugging Face tokenizer: {model_name}...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        print(f"Loading Hugging Face model: {model_name}...")
        # Load only the base model structure if not fine-tuning and to save memory.
        model = AutoModel.from_pretrained(model_name)
        print("Hugging Face model loaded successfully.")
        # Here you could typically use the tokenizer and model for NLP tasks:
        # Example: inputs = tokenizer(command, return_tensors="pt")
        #          outputs = model(**inputs)
        #          # Process 'outputs' to extract intent/entities
    except Exception as e:
        # Handle errors during model loading
        print(f"Warning: Could not load Hugging Face model '{model_name}'. Error: {e}")
        print("Proceeding with keyword-based parsing only.")
        transformers_available = False # Treat as unavailable if loading fails
else:
    # Message if the library was not available from the start
    print("\nProceeding with keyword-based parsing only as 'transformers' library is unavailable.")


# --- Simple Parsing Function ---
def parse_command_simple(command):
    """
    Parses a natural language command using simple keyword matching.
    This is a simplified implementation and acts as a placeholder for
    more advanced AI-based parsing.

    Args:
        command (str): The text command provided by the user.

    Returns:
        dict or None: A dictionary containing 'intent', 'device_type', 'value'
                      if the command is recognized, otherwise None.
                      Ex: {'intent': 'set_state', 'device_type': 'light', 'value': 'on'}
                      Ex: {'intent': 'get_status', 'device_type': 'thermostat'}
                      Ex: {'intent': 'get_all_status'}
    """
    command_lower = command.lower() # Convert command to lowercase for consistency

    # --- Intent and Device/Value Identification ---

    # General status requests
    if "status of all devices" in command_lower or "status all" in command_lower:
         return {'intent': 'get_all_status'} # Specific intent for global status

    # Specific status requests or control commands
    if "light" in command_lower:
        device = 'light'
        if "status" in command_lower:
            return {'intent': 'get_status', 'device_type': device}
        if "turn on" in command_lower or "switch on" in command_lower:
            return {'intent': 'set_state', 'device_type': device, 'value': 'on'}
        if "turn off" in command_lower or "switch off" in command_lower:
            return {'intent': 'set_state', 'device_type': device, 'value': 'off'}

    if "fan" in command_lower:
        device = 'fan'
        if "status" in command_lower:
             return {'intent': 'get_status', 'device_type': device}
        if "turn on" in command_lower or "switch on" in command_lower:
             return {'intent': 'set_state', 'device_type': device, 'value': 'on'}
        if "turn off" in command_lower or "switch off" in command_lower:
            return {'intent': 'set_state', 'device_type': device, 'value': 'off'}
        # Check for speed keywords
        # Access Fan.VALID_SPEEDS directly as the class is defined above in this scope
        for speed in Fan.VALID_SPEEDS:
            if speed in command_lower:
                 return {'intent': 'set_speed', 'device_type': device, 'value': speed}

    if "thermostat" in command_lower or "temperature" in command_lower:
        device = 'thermostat'
        if "status" in command_lower or "what is the" in command_lower or "current" in command_lower:
             return {'intent': 'get_status', 'device_type': device}
        # Try to extract a numerical value for temperature using regex
        match = re.search(r'(\d+)', command_lower) # Search for a sequence of digits
        if match:
             temp_value = int(match.group(1)) # Extract the found number
             return {'intent': 'set_temp', 'device_type': device, 'value': temp_value}

    # If no match was found, the command is not recognized
    return None # Indicate command was not understood

print("\nParsing function defined.")



Loading Hugging Face tokenizer: Musixmatch/umberto-wikipedia-uncased-v1...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/801k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

Loading Hugging Face model: Musixmatch/umberto-wikipedia-uncased-v1...


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Hugging Face model loaded successfully.

Parsing function defined.


In [8]:
# -----------------------------------------------------------------------------
# Block 3: Command Execution Logic (from executor.py)
# -----------------------------------------------------------------------------

def execute_command(parsed_command, devices):
    """
    Executes the action specified in the parsed command on the appropriate device.
    (Adapted version for single scope - does not import classes from devices).

    Args:
        parsed_command (dict or None): The dictionary returned by the parser
                                      (e.g., {'intent': 'set_state', ...})
                                      or None if the command wasn't understood.
        devices (dict): A dictionary mapping device types ('light', 'fan', ...)
                        to their instances (Light(), Fan(), ... objects).
                        Ex: {'light': light_obj, 'fan': fan_obj, ...}

    Returns:
        str: A feedback message for the user describing the outcome
             of the operation or an error message.
    """
    # If the parser didn't understand the command
    if not parsed_command:
        return "Sorry, I didn't understand that command."

    # Extract information from the parsed command dictionary
    intent = parsed_command.get('intent')
    device_type = parsed_command.get('device_type') # Ex: 'light', 'fan', 'thermostat'
    value = parsed_command.get('value')             # Ex: 'on', 'off', 'medium', 25

    # --- Execution Logic ---

    # Special case: request status of all devices
    if intent == 'get_all_status':
        # Collect status from all devices in the 'devices' dictionary
        all_statuses = [d.get_status() for d in devices.values()]
        # Join the statuses into a single string, separated by newlines
        return "\n".join(all_statuses)

    # Find the specific device instance requested
    device_instance = None
    if device_type:
        device_instance = devices.get(device_type)
        # If no device of the requested type exists
        if not device_instance:
            return f"No device of type '{device_type}' found."

    # --- Execute action based on intent ---

    if intent == 'get_status':
        # If a specific instance was requested
        if device_instance:
            # Call the get_status() method of the found device
            return device_instance.get_status()
        else:
            # If intent was 'get_status' but device_type was missing
            return "Please specify which device's status you want (e.g., 'light status')."

    elif intent == 'set_state': # Turn On/Off
        if not device_instance: return "Which device do you want to control?"
        if value == 'on':
            # Check if the device has the 'turn_on' method
            if hasattr(device_instance, 'turn_on'):
                return device_instance.turn_on() # Call the method
            else:
                return f"{device_instance.name} cannot be turned on/off directly like this."
        elif value == 'off':
             # Check if the device has the 'turn_off' method
            if hasattr(device_instance, 'turn_off'):
                return device_instance.turn_off() # Call the method
            else:
                 return f"{device_instance.name} cannot be turned on/off directly like this."
        else:
             return f"Unknown state '{value}'." # If the value is neither 'on' nor 'off'

    elif intent == 'set_speed': # Set speed (only for fan)
        # Use isinstance to check the type, as classes are defined above
        if isinstance(device_instance, Fan) and hasattr(device_instance, 'set_speed'):
            # Call the method, passing the value (ensuring it's a string if needed)
            return device_instance.set_speed(str(value))
        else:
            # Message if trying to set speed on an unsuitable device
            return "Speed can only be set for the fan."

    elif intent == 'set_temp': # Set temperature (only for thermostat)
         # Use isinstance to check the type
        if isinstance(device_instance, Thermostat) and hasattr(device_instance, 'set_temperature'):
            # Call the method passing the value
            return device_instance.set_temperature(value)
        else:
            # Message if trying to set temperature on an unsuitable device
            return "Temperature can only be set for the thermostat."

    else:
        # Fallback if the intent is not one of the handled ones
        return "Sorry, I couldn't execute that command (unknown intent)."

print("Execution function defined.")

Execution function defined.


# 4: Main Logic and CLI (from main.py)

In [9]:

def run_smart_home_cli():
    """
    Starts and manages the Smart Home CLI application within the cell.
    (Previously was the main() function).
    """
    # --- Device Initialization ---
    # The Light, Fan, Thermostat classes are defined above in this cell
    devices = {
        "light": Light(),      # Create a Light instance
        "fan": Fan(),          # Create a Fan instance
        "thermostat": Thermostat() # Create a Thermostat instance
    }

    # --- Start User Interface (CLI) ---
    print("\n--- Smart Home Control (Jupyter/Colab) ---")
    print("Enter commands (e.g., 'turn on light', 'set fan to medium', 'temperature?', 'status all').")
    print("Type 'quit' or 'exit' to end.")
    print("-" * 40)

    # Print initial status of all devices
    print("Initial Device Status:")
    # The parse_command_simple and execute_command functions are defined above
    initial_status_command = parse_command_simple("status all") # Or create dict directly
    print(execute_command(initial_status_command, devices))
    print("-" * 40)

    # --- Main Application Loop ---
    # Keep asking for user input until they decide to exit.
    while True:
        try:
            # Prompt the user for input, showing "> "
            command = input("> ").strip() # .strip() removes leading/trailing whitespace

            # Check if the user wants to exit
            if command.lower() in ["quit", "exit"]:
                print("Exiting Smart Home Control.")
                break # Exit the while loop

            # If the user just pressed Enter without typing anything, continue loop
            if not command:
                continue

            # --- Command Processing Flow ---
            # 1. Parse Command: Interpret the text command.
            #    (Uses the simple parser defined above)
            #    If using AI: parsed_command = parse_command_ai(command, model, tokenizer)
            parsed_command = parse_command_simple(command)

            # 2. Execute Command: Perform the action on the correct device.
            #    Pass the parsing result and the devices dictionary.
            feedback = execute_command(parsed_command, devices)

            # 3. User Feedback: Print the result of the action.
            print(feedback)

        # Handle interruption (e.g., Stop button in Colab/Jupyter)
        except KeyboardInterrupt:
             print("\nManual interruption. Exiting.")
             break
        # Handle end of input (less common in notebooks, but for safety)
        except EOFError:
             print("\nInput ended. Exiting.")
             break
        # Handle other unexpected errors
        except Exception as e:
             print(f"An unexpected error occurred: {e}")
             # You might want to break the loop in case of a serious error
             # break

print("CLI function defined.")

CLI function defined.


In [ ]:
# CLI the interface
run_smart_home_cli()


--- Smart Home Control (Jupyter/Colab) ---
Enter commands (e.g., 'turn on light', 'set fan to medium', 'temperature?', 'status all').
Type 'quit' or 'exit' to end.
----------------------------------------
Initial Device Status:
Living Room Light: OFF
Ceiling Fan: OFF
Main Thermostat: Set to 22°C
----------------------------------------
> status light
Living Room Light: OFF
> status ceiling fan
Ceiling Fan: OFF
> status temperature?
Main Thermostat: Set to 22°C
> what is the temperature?
Main Thermostat: Set to 22°C
> turn ceiling fan and lights on
Sorry, I didn't understand that command.
> turn on the light
Living Room Light is now ON.
> set the temperature to 28
Main Thermostat temperature set to 28°C.
> set the fan to medium
Ceiling Fan turned ON and speed set to medium.
> status all
Living Room Light: ON
Ceiling Fan: ON (Speed: medium)
Main Thermostat: Set to 28°C
